In [2]:
import dotenv
import os
import subprocess

In [3]:
dotenv.load_dotenv()

True

In [4]:
# Load the Mapbox access token and username from environment variables
MAPBOX_UPLOAD_ACCESS_TOKEN = os.getenv("MAPBOX_UPLOAD_ACCESS_TOKEN")
MAPBOX_USERNAME = os.getenv("MAPBOX_USERNAME")


In [5]:
import json
import os

def set_geojson_precision_and_trim_write(n_coords: int, n_trim: int, file_path: str) -> tuple:

    with open(file_path, 'r') as f:
        data = json.load(f)
    
    def set_precision_and_trim(coords, n):
        if isinstance(coords, list):
            return [set_precision_and_trim(c, n) for c in coords]
        elif isinstance(coords, dict):
            return {k: set_precision_and_trim(v, n) for k, v in coords.items()}
        elif isinstance(coords, float):
            return round(coords, n)
        else:
            if isinstance(coords, str) and coords.isdigit():
                return coords  # Don't trim strings that are only digits
            try:
                return float(coords)  # Leave other numbers untouched
            except (ValueError, TypeError):
                return coords
    
    for i,feature in enumerate(data['features']):
        for key in data['features'][i]["properties"].keys():
            if type(data['features'][i]["properties"][key]) == float:
                data['features'][i]["properties"][key] = round(data['features'][i]["properties"][key], n_trim)
            else:
                continue
            
        data['features'][i]["geometry"]["coordinates"] = set_precision_and_trim(data['features'][i]["geometry"]["coordinates"], n_coords)

    # Write the modified data to a new file with the name appended with "_trimmed"
    path, ext = os.path.splitext(file_path)
    new_file_path = path + '_trimmed' + ext
    with open(new_file_path, 'w') as f:
        json.dump(data, f)
    
    return os.path.basename(new_file_path), new_file_path

In [6]:

def upload (MAPBOX_USERNAME, mytileset, file_path, MAPBOX_UPLOAD_ACCESS_TOKEN):

    os.environ["MAPBOX_ACCESS_TOKEN"] = MAPBOX_UPLOAD_ACCESS_TOKEN

    # Define the CLI commands as strings
    upload_cmd = f"tilesets upload-source {MAPBOX_USERNAME} {mytileset} {file_path}"
    view_cmd = f"tilesets view-source {MAPBOX_USERNAME} {mytileset}"
    # Run each command using subprocess
    subprocess.run(upload_cmd.split())
    view_output = subprocess.run(view_cmd.split(), capture_output=True)
        
    source = json.loads(view_output.stdout.decode())["id"]

    data = {
        "version": 1,
        "layers": {
        "trees": {
            "source": source,
            "minzoom": 9,
            "maxzoom": 16
            }
        }
    }
    
    with open(f'./out/{mytileset}.json', 'w') as f:
        json.dump(data, f)
    
    create_cmd = f'tilesets create {MAPBOX_USERNAME}.{mytileset} --recipe ./out/{mytileset}.json --name "{mytileset}"'
    publish_cmd = f"tilesets publish {MAPBOX_USERNAME}.{mytileset}"

    subprocess.run(create_cmd.split())
    publish_output = subprocess.run(publish_cmd.split(), capture_output=True)
    
    return publish_output.stdout.decode()


In [7]:
# Define the name and filepath of the GeoJSON file to upload
file_name = "buildings.geojson"
file_path = "./processor/out/singapore/final/buildings.geojson"

mytileset = "singapore-buildings"

file_name, file_path = set_geojson_precision_and_trim_write(7, 3, file_path)

In [8]:
upload(MAPBOX_USERNAME, mytileset, file_path, MAPBOX_UPLOAD_ACCESS_TOKEN)

upload progress


In [ ]:
# Define the name and filepath of the GeoJSON file to upload
file_name = "tessellation.geojson"
file_path = "./processor/out/singapore/final/tessellation.geojson"
mytileset = "singapore-tessellation"

file_name, file_path = set_geojson_precision_and_trim_write(7, 3, file_path)

In [ ]:
# def split_json(input_file: str) -> None:
    
#     # Define the file pattern to search for
#     file_pattern = f'./processor/out/singapore/final/output_{mytileset}*.geojson'

#     # Get a list of files that match the pattern
#     file_list = glob.glob(file_pattern)

#     # Loop through the file list and delete each file
#     for file_path in file_list:
#         os.remove(file_path)

#     # Maximum size of each output GeoJSON file (in bytes)
#     max_size = 4 * 1024 * 1024  # 4 MB

#     # Load the input GeoJSON file as a Python dictionary
#     with open(input_file, 'r') as f:
#         data = json.load(f)

#     # Extract the list of features from the GeoJSON dictionary
#     features = data['features']

#     # Initialize variables for the current file and its size
#     current_file = 0
#     current_size = 0
    
#     feature_collection = {
#         'type': 'FeatureCollection',
#         'features': []
#     }

#     # Loop through each feature and write it to a separate file
#     for i, feature in enumerate(features):
#         # Create a new GeoJSON dictionary with only the current feature
#         # Get the size of the serialized GeoJSON string
#         feature_size = len(str(feature))

#         # If adding the current feature to the current file would make it too large,
#         # start a new file
#         if current_size + feature_size > max_size:
#             current_file += 1
#             current_size = 0
            
#                     # Write the GeoJSON string to a file
#             filename = f'output_{mytileset}_{current_file}.geojson'
#             with open(f"./processor/out/singapore/final/{filename}", 'w') as f:
#                 json.dump(feature_collection, f)
                
#             feature_collection["features"] = []
            
#         feature_collection['features'].append(feature)

#         # Update the current file size
#         current_size += feature_size
    
#     filename = f'output_{mytileset}_{current_file}.geojson'
#     with open(f"./processor/out/singapore/final/{filename}", 'w') as f:
#         json.dump(feature_collection, f)

In [ ]:
# split_json(file_path)

In [ ]:
# with open(file_path, 'r') as f:
#     data = json.load(f)

# data = data['features']

In [ ]:
# def upload (MAPBOX_UPLOAD_ACCESS_TOKEN, MAPBOX_USERNAME, mytileset):
#   dataset = {
#     "name": mytileset
#   }

#   # Construct the curl command as a list of arguments
#   curl_command = [
#       "curl",
#       "-X",
#       "POST",
#       f"https://api.mapbox.com/datasets/v1/{MAPBOX_USERNAME}?access_token={MAPBOX_UPLOAD_ACCESS_TOKEN}",
#       "-d",
#       json.dumps(dataset),
#       "--header",
#       "Content-Type:application/json"
#   ]

#   # Run the curl command using subprocess
#   process = subprocess.Popen(curl_command,
#                       stdout=subprocess.PIPE, 
#                       stderr=subprocess.PIPE)
#   stdout, stderr = process.communicate()

#   response = json.loads(stdout)

#   id = response["id"]

#   # Define the file pattern to search for
#   file_pattern = f'./processor/out/singapore/final/output_{mytileset}_*.geojson'

#   # Get a list of files that match the pattern
#   file_list = glob.glob(file_pattern)

#   for i, feature in enumerate(data):
#       # Construct the curl command as a list of arguments
#       curl_command = [
#           "curl",
#           "-X",
#           "PUT",
#           f"https://api.mapbox.com/datasets/v1/{MAPBOX_USERNAME}/{id}/features/{i}?access_token={MAPBOX_UPLOAD_ACCESS_TOKEN}",
#           "-H",
#           "Content-Type:application/json",
#           "-d",
#           json.dumps(feature),
#           "-v"
#       ]

#       # Run the curl command using subprocess
#       process = subprocess.Popen(curl_command,
#                           stdout=subprocess.PIPE, 
#                           stderr=subprocess.PIPE)
#       stdout, stderr = process.communicate()

#       response = json.loads(stdout)

#   return id


In [ ]:
# dataset_id = upload(MAPBOX_UPLOAD_ACCESS_TOKEN, MAPBOX_USERNAME, mytileset)

In [ ]:
# def upload (MAPBOX_UPLOAD_ACCESS_TOKEN, MAPBOX_USERNAME, file_name, file_path, mytileset):

#     # Construct the curl command as a list of arguments
#     curl_command = [
#         "curl",
#         "-X",
#         "POST",
#         f"https://api.mapbox.com/uploads/v1/{MAPBOX_USERNAME}/credentials?access_token={MAPBOX_UPLOAD_ACCESS_TOKEN}"
#     ]

#     # Run the curl command using subprocess
#     process = subprocess.Popen(curl_command,
#                         stdout=subprocess.PIPE, 
#                         stderr=subprocess.PIPE)
#     stdout, stderr = process.communicate()
    
#     response = json.loads(stdout)
#     access_key_id = response["accessKeyId"]
#     secret_access_key = response["secretAccessKey"]
#     session_token = response["sessionToken"]
#     bucket = response["bucket"]
#     key = response["key"]
    
#     print(response)

#     # Construct the curl command as a list of arguments
#     curl_command = [
#         "curl",
#         "-X",
#         "POST",
#         "-H",
#         "Content-Type: application/json", 
#         "-H",
#         "Cache-Control: no-cache",
#         "-d"
#         f"'{"tileset": "{MAPBOX_USERNAME}.mytileset","url": "mapbox://datasets/{MAPBOX_USERNAME}/{dataset_id}","name": "{mytileset}" }'",
#         f"https://api.mapbox.com/uploads/v1/{MAPBOX_USERNAME}?access_token={MAPBOX_UPLOAD_ACCESS_TOKEN}"
#     ]

#     # Run the curl command using subprocess
#     process = subprocess.Popen(curl_command,
#                         stdout=subprocess.PIPE, 
#                         stderr=subprocess.PIPE)
#     stdout, stderr = process.communicate()

#     response = json.loads(stdout)
#     print(response)

#     # Construct the curl command as a list of arguments
#     curl_command = [
#         "curl",
#         f"https://api.mapbox.com/uploads/v1/{MAPBOX_USERNAME}?reverse=true&limit=10&access_token={MAPBOX_UPLOAD_ACCESS_TOKEN}"
#     ]

#     # Run the curl command using subprocess
#     process = subprocess.Popen(curl_command,
#                         stdout=subprocess.PIPE, 
#                         stderr=subprocess.PIPE)
#     stdout, stderr = process.communicate()

#     response = json.loads(stdout)
#     print(response)

In [ ]:
# upload (MAPBOX_UPLOAD_ACCESS_TOKEN, MAPBOX_USERNAME, file_name, file_path, mytileset)

In [ ]:
# print(file_name, file_path, mytileset)
# upload (MAPBOX_UPLOAD_ACCESS_TOKEN, MAPBOX_USERNAME, file_name, file_path, mytileset)